
# First In-class Project: Coding a Rank-order Assignment Algorithm
## TEAM INFORMATION:
* Naina Misra
* Rex Wang
* Phoebe Wu


## CONTRIBUTIONS:
* Naina - Conducted research, helped develop code for Hungarian algorithm, developed the code to output which doctors are matched to each hospital, edited code, wrote introduction
* Rex - Conducted research, helped develop code handling input and printing output
* Phoebe - Conducted research and found resources, helped brainstorm assignment system, commented code, wrote Algorithm Implemented


## ASSUMPTIONS:
* Hospitals do not participate in ranking doctors
* Doctors will always rank every hospital
* Input matrix will be i = doctors, j = hospital
* Capacity for each hospital will be mentioned in capacities, and be non-negative


## ALGORITHM IMPLEMENTED:

We need to assign doctors to hospitals based on their ranked preference.

The doctors rank their preference of hospitals with (1) being the number one preference and larger numbers representing lower preference.

There is a capacity for the number of doctors for each hospital and a doctor can only be assigned to one hospital.

To solve this problem, we choose to work with the Hungarian Algorithm, which is also known as the Munkres Algorithm.

This algorithm provides a way to match a doctor to a hospital while minimizing the cost. In our system, the cost is defined based on the doctor's preference.
Therefore, the higher the rank of the hospital (1 being the highest), the lower the cost.

The preferences data was represented by a matrix with the rows being the doctor's preferences of each hospital and the hospital as columns.
When a hospital had multiple slots open, the matrix was expanded to have multiple columns representing the hospital to reflect the capacity.
The Munkres Algorithm requires a square matrix.

However, sometimes, the number of doctors and hospitals may be unequal.
Therefore, the matrix was padded with dummy doctors or hospital slots to ensure the algorithm runs smoothly while the doctors that were
unmatched remained unassigned.

The Munkres library in Python was used to compute the assignments.
The algorithm produces a list of doctors matched with hospitals as well as doctors that did not have a match (unmatched).


## RESOURCES USED:
### Hungarian algorithm original paper
Harold W. Kuhn (1955). The Hungarian Method for the Assignment Problem. Naval Research Logistics Quarterly, 2(1–2), 83–97.

### Reference code
https://software.clapper.org/munkres/


In [ ]:
from munkres import Munkres
import numpy as np

#function to assign doctors to hospital based on preference
def assign_doctors(ranks, capacities):
    """
    Assign doctors to hospitals using the Hungarian algorithm.

    Parameters
    ----------
    ranks : 2D list or ndarray
        ranks[i][j] = doctor i's rank of hospital j (1 = best, higher = less preferred).
    capacities : list[int]
        capacity[j] = number of slots available at hospital j; should be non-negative

    Returns
    -------
    assignments : list[int]
        assignments[i] = hospital index assigned to doctor i.
    """
    ##Error statments: Check if inputs are valid

    # Check if the rank input is a list of lists of integers
    is_2d = (isinstance(ranks, list)                        # the input is a list ...
      and all(isinstance(row, list) for row in ranks)             # ... of lists ...
      and all(all(isinstance(rank, int) for rank in row) for row in ranks) # ... of integers
    )
    if not is_2d:
      raise ValueError(f"Expected list of lists of positive integers for ranks")

    # Check if the lists of preferences have the same length
    same_len = len({len(row) for row in ranks}) == 1
    if not same_len:
      raise ValueError(f"Expected each doctor to rank the same number of hospitals")

    # Check if all rankings are positive integers
    all_positive = all(all(rank > 0 for rank in row) for row in ranks)
    if not all_positive:
      raise ValueError(f"Expected ranks to be positive")

    # Check if all capacities are non-negative integers
    valid_cap = all(isinstance(c, int) for c in capacities) and all(c >= 0 for c in capacities)
    if not valid_cap:
      raise ValueError(f"Expected capacities to be non-negative integers")

    # Check if there is a capacity for each hospital
    num_cap = len(capacities) == len(ranks[0])
    if not num_cap:
      raise ValueError(f"Expected one capacity provided for each hospital")


    # Convert ranks to a NumPy array
    ranks = np.array(ranks)
    n_doctors, n_hospitals = ranks.shape                  #gets the number of doctors and hospitals

    # Expand hospitals into slots
    expanded_cols = []                                    #stores the duplicated columns of the hospital
    slot_to_hospital = []                                 #keeps track of which slot is for which hospital
    for j, cap in enumerate(capacities):                  # j = hospital index, cap = capacity of hospital j
        for _ in range(cap):                              # repeats the "cap = capacity" times
            expanded_cols.append(ranks[:, j][:, None])    # adds hospital j's column to the expanded columns
            slot_to_hospital.append(j)                    #records which hospital this slot is for

    cost_matrix = np.hstack(expanded_cols)                #combines expanded columns into matrix

    # If there are more slots than doctors, pad rows with zeros
    if cost_matrix.shape[1] > n_doctors:
        pad_rows = cost_matrix.shape[1] - n_doctors       #gives you the number of rows needed
        pad = np.zeros((pad_rows, cost_matrix.shape[1]))  #pads the unfilled rows with 0
        cost_matrix = np.vstack([cost_matrix, pad])       #adds the pads to the matrix

    # Similarly, if there are more slots than doctors, pad columns with zeros
    if n_doctors > cost_matrix.shape[1]:
        pad_cols = n_doctors - cost_matrix.shape[1]       #gives you the number of columns needed
        pad = np.zeros((cost_matrix.shape[0], pad_cols))  #pads the unfilled columns with 0
        cost_matrix = np.hstack([cost_matrix, pad])       #adds the pads to the matrix

    # If there are more doctors than slows, pad columns with large numbers

    # Convert to NumPy array to list of lists for Munkres
    cost_matrix = cost_matrix.tolist()

    #Runs algorithm on matrix
    m = Munkres()
    indexes = m.compute(cost_matrix)

    #Build doctor to hospital assignments
    assignments = [-1] * n_doctors                        #all doctors are unassigned (-1) for the initial assignment
    for row, col in indexes:                              #loops through each pair
        if row < n_doctors and col < len(slot_to_hospital):                               #ignore padded dummy doctors
            assignments[row] = slot_to_hospital[col]      #Map each doctor to its assigned hospital

    #Print final assignment for doctors to hospitals
    for doctor, hospital in enumerate(assignments):       #Loops through doctors and assigned hospital
      if hospital >= 0:                                   #prints match if there is a match
        print(f"Doctor {doctor} is matched to hospital {hospital}")
      else:                                               #Unmatched doctors have an index of -1, because when doctors go unassigned their index stays the same as originally assigned
        print(f"Doctor {doctor} wasn't assigned to a hospital :(")
    return assignments                                    # returns final list





# -------------------
# Input your matrix in ranks and chosen capacities here
# Example usage given
# -------------------
if __name__ == "__main__":
    # ranks[i][j] = doctor i’s rank for hospital j
    ranks = [
        [2, 1],
        [2, 3, 1],
        [3, 2, 1]
    ]

    capacities = [1, 1]                                    #Hospital 0 has 1 slot, hospital 1 has 1 (total 2 hospitals)

    assignments = assign_doctors(ranks, capacities)        #Calls function to run assignment algorithm

ValueError: Expected each doctor to rank the same number of hospitals